In [1]:
import numpy
import pandas
from matplotlib import pyplot

In [ ]:
# データセットの読み込み
filename = "airdata.csv"
data = pandas.read_csv(filename, header=0, index_col=0).T
data.index = pandas.read_csv(filename, header=None, index_col=0).iloc[0].values
xmax, xmin = data.columns.max(), data.columns.min()
data.T.plot()
pyplot.gca().invert_xaxis()
pyplot.gca().legend_ = None
pyplot.show()

In [ ]:
# 空気スペクトルの読み込み
airfilename = "air.csv"
air = pandas.read_csv(airfilename, header=None, index_col=0).squeeze()
air.index.name = ''
air.plot()
pyplot.gca().invert_xaxis()
pyplot.show()

In [ ]:
# 横軸範囲 / change horizontal region
xmax, xmin = 4000, 1100  # 範囲
buff = data.iloc[:, (xmin <= data.columns) & (data.columns <= xmax)]
buff.T.plot()
pyplot.gca().invert_xaxis()
pyplot.gca().legend_ = None
pyplot.show()
airbuff = air.iloc[(xmin <= air.index) & (air.index <= xmax)]
airbuff.plot()
pyplot.gca().invert_xaxis()
pyplot.show()
#data, air = buff, airbuff

In [ ]:
# 水蒸気補正に適した領域の指定
xmax, xmin = 4000, 3700  # 領域指定（データセットのスペクトルに信号がない領域を選ぶ）
buff = data.iloc[:, (xmin <= data.columns) & (data.columns <= xmax)]
buff = (buff.T - buff.iloc[:,-1]).T
x = buff.columns
from scipy.spatial import ConvexHull
for i in range(len(buff)):
	y = buff.iloc[i].values
	v = ConvexHull(list(zip(x, y))).vertices
	v = numpy.roll(v, -v.argmin())
	v = v[:v.argmax()]
	buff.iloc[i] = buff.iloc[i] - numpy.interp(x, x[v], y[v])
buff.T.plot()
pyplot.gca().invert_xaxis()
pyplot.gca().legend_ = None
pyplot.show()
air = air - air.min()
airbuff = air.iloc[(xmin <= air.index) & (air.index <= xmax)]
x = airbuff.index
y = airbuff.values
v = ConvexHull(list(zip(x, y))).vertices
v = numpy.roll(v, -v.argmin())
v = v[:v.argmax()]
airbuff = airbuff - numpy.interp(x, x[v], y[v])
airbuff.plot()
pyplot.gca().invert_xaxis()
pyplot.show()

In [ ]:
# 水蒸気補正
sub = data.values - numpy.array([buff.T.sum() / airbuff.sum()]).T * air.values
buff2 = pandas.DataFrame(sub, index=data.index, columns=data.columns)
buff2.T.plot(legend=None, xlim=[data.columns.max(), data.columns.min()])
pyplot.show()
data = buff2

In [ ]:
# 横軸範囲 / change horizontal region
xmax, xmin = 4000, 3000  # 範囲
buff = data.iloc[:, (xmin <= data.columns) & (data.columns <= xmax)]
buff.T.plot()
pyplot.gca().invert_xaxis()
pyplot.gca().legend_ = None
pyplot.show()
#data = buff

In [8]:
#data.T.to_csv(filename[:-4] + "_.csv")